In [ ]:
# External 
import scipy as scp
import tensorflow as tf
from scipy.stats import gamma
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
import random
import multiprocessing as mp
import psutil
import pickle 
import os
import re

# Own
import kde_training_utilities as kde_util
import kde_class as kde
import ddm_data_simulation as ddm_sim

In [ ]:
# Simulate: RACE MODEL
def my_boundary_race(t = 0, height = 1):
    return height

# Get samples
my_samples_race  = ddm_sim.race_model(v = np.array([1, 0.5, 0, 0, 0], ndmin = 2).T,
                                      w = np.array([0, 0, 0, 0, 0], ndmin = 2).T, 
                                      s = 1,
                                      delta_t = 0.001,
                                      max_t = 20,
                                      n_samples = 3000,
                                      print_info = True,
                                      boundary_fun = my_boundary_race,
                                      height = 2)

In [ ]:
# Define boundary functions
def gamma_shape_scale(t = 1,
                      shape = 1.01,
                      scale = 1):
    return gamma.pdf(t, a = shape, scale = scale)

In [ ]:
# Exponential decay (c2) with decay starting point (c1)
def exp_c1_c2(t = 1, 
              c1 = 1,
              c2 = 1):
    
    b = np.exp(- c2*(t-c1))
    
    if t >= c1:
        
        return b
        
    else:
        return 1

In [ ]:
# Linear collapse
def linear_collapse(t = 1, 
                    node = 1,
                    theta = 1):
    if t >= node:
        return (t - node) * (- np.sin(theta) / np.cos(theta))
    else:
        return 0

In [ ]:
# Logistic
def logistic_bound(t = 1,
                   node = 1,
                   k = 1,
                   midpoint = 1,
                   max_val  = 3):
    
    return - (max_val / (1 + np.exp(- k * ((t - midpoint)))))


In [ ]:
# Generalized logistic bound
def generalized_logistic_bnd(t = 1, 
                             B = 2.,
                             M = 3.,
                             v = 0.5):
    return 1 - (1 / np.power(1 + np.exp(- B * (t - M)), 1 / v))

In [ ]:
# Plot boundary multiplicative
t_s = np.arange(0, 20, 0.01)
boundary = np.zeros((len(t_s),2))

a = 3
node = 1
for v in np.arange(0.1, 1.11, 0.5):
    for B in np.arange(3, 4, 1):
        cnt = 0
        for i in t_s:
            b = a * generalized_logistic_bnd(t = i,
                                             B = 1,
                                             v = v, 
                                             M = 1)   #B = B)
            #print(b)
            if b <= 0:
                boundary[cnt, 0] = 0
                boundary[cnt, 1] = 0
            else: 
                boundary[cnt, 0] = b
                boundary[cnt, 1] = - boundary[cnt, 0]
            cnt += 1

        plt.plot(t_s, boundary[:, 0])
        plt.plot(t_s, boundary[:, 1]) 

In [ ]:
# Plot boundary additive
t_s = np.arange(0, 1.5, 0.01)
boundary = np.zeros((len(t_s),2))

a = 1
node = 1
for a in range(1, 2, 1):
    for theta in np.arange(0.01, (1/2 * np.pi), 0.1):
        cnt = 0
        for i in t_s:
            b = a + linear_collapse(t = i, node = node, theta = theta)
            if b <= 0:
                boundary[cnt, 1] = 0
                boundary[cnt, 0] = 0
            else: 
                boundary[cnt, 1] = b
                boundary[cnt, 0] = - b
            cnt += 1

        plt.plot(t_s, boundary[:, 0])
        plt.plot(t_s, boundary[:, 1]) 

In [ ]:
# Simulations
my_samples_flexb  = ddm_sim.ddm_flexbound_simulate(v = 0.5,
                                                   a = 5,
                                                   w = 0.5, 
                                                   s = 1,
                                                   delta_t = 0.001,
                                                   max_t = 20,
                                                   n_samples = 1000,
                                                   print_info = True,
                                                   boundary_fun = linear_collapse,
                                                   boundary_multiplicative = False,
                                                   boundary_params = {'node': 0.1, 'theta': 1}
                                                   )